In [ ]:
import os
import boto3
import pandas as pd
import numpy as np
import sagemaker as sm
from io import StringIO

bucket = '1s-gary'
data_key = 'aiml-blackbelt-2021/rawdata/PS_20174392719_1491204439457_log.csv'
data_model_drift_key = 'aiml-blackbelt-2021/rawdata/PS_20174392719_1491204439457_log_model_drift.csv'

s3_client = boto3.client('s3')
csv_obj = s3_client.get_object(Bucket=bucket, Key=data_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))
df['deltaOrig']=df['newbalanceOrig']-df['oldbalanceOrg']
df['deltaDest']=df['newbalanceDest']-df['oldbalanceDest']
# get lags; see analysis below
df.loc[:,'amount_previous'] = df.amount.shift(1)
df.loc[:,'type_previous'] = df.type.shift(1)
df.drop(['nameOrig','nameDest','isFlaggedFraud'],axis=1,inplace=True)
